## Description
This code replicates the model discussed in the following research
> A. Chadha, R. Dara and Z. Poljak, "Convolutional Classification of Pathogenicity in H5 Avian Influenza Strains," 2019 18th IEEE International Conference On Machine Learning And Applications (ICMLA), Boca Raton, FL, USA, 2019, pp. 1570-1577.

- Within research 1202 HP sequences, 1167 LP sequences were used which were gathered from various sources such as https://www.fludb.org.  

- This code has yet to collect relevant number of data and works with only 133 HP sequences and 750 LP sequences  
- These sequences were collected from https://www.fludb.org only  
- They are all HA segments of H5 avian influenza virus of various kinds.  
- These HA segments are aligned using MUSCLE (Multiple Sequence Comparison by Log-Expectation) algorithm, available [here](https://www.fludb.org/brc/msa.spg?method=ShowCleanInputPage&decorator=influenza)  


In [1]:
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from Bio import SeqIO
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv1D, MaxPool1D, Flatten, Dense

In [2]:
# Load data in records, and outputs in y
records = []
y = []
with open('./all-aligned-seq.fasta') as handle:
    for record in SeqIO.parse(handle, 'fasta'):
        attr = record.id.split('|')
        records.append(record)
        if(attr[3] == 'Yes'):
            y.append(1)
        elif(attr[3] == 'No'):
            y.append(0)
print('Highly Pathogenic cases:', y.count(1))
print('Low Pathogenic cases:', y.count(0))

Highly Pathogenic cases: 133
Low Pathogenic cases: 750


In [3]:
# preprocess input data
sequence = np.array(records)
y = tf.one_hot(np.array(y), 2)
le = LabelEncoder()
seqEncoded = np.empty_like(sequence, dtype='int')
for i, seq in enumerate(sequence):
    seqEncoded[i] = le.fit_transform(seq)

oneHotSeq = tf.one_hot(seqEncoded, depth=21).numpy()

# remove alignment character's one hot
for seq in oneHotSeq:
    for protein in seq:
        if protein[0] == 1:
            protein[0] = 0
print('(samples, proteins, one-hot-encoding) ::',oneHotSeq.shape)

(samples, proteins, one-hot-encoding) :: (883, 576, 21)


In [4]:
# building Model
model = Sequential()

model.add(Conv1D(20,
                 kernel_size=2,
                 strides=2,
                 activation='relu',
                 input_shape=(oneHotSeq.shape[1], oneHotSeq.shape[2],)))
model.add(MaxPool1D(pool_size=2,
                    strides=2,
                    padding='valid'))
model.add(Flatten())
model.add(Dense(2,activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 288, 20)           860       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 144, 20)           0         
_________________________________________________________________
flatten (Flatten)            (None, 2880)              0         
_________________________________________________________________
dense (Dense)                (None, 2)                 5762      
Total params: 6,622
Trainable params: 6,622
Non-trainable params: 0
_________________________________________________________________


In [5]:
# compiling model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [13]:
# training model
model.fit(
    oneHotSeq, y,
    batch_size=32,
    epochs=10,
    validation_split=0.3,
    shuffle=True,
)

Train on 618 samples, validate on 265 samples
Epoch 1/10
618/618 [==============================] - 0s 151us/sample - loss: 0.0044 - accuracy: 1.0000 - val_loss: 0.0031 - val_accuracy: 1.0000
Epoch 2/10
618/618 [==============================] - 0s 149us/sample - loss: 0.0031 - accuracy: 1.0000 - val_loss: 0.0011 - val_accuracy: 1.0000
Epoch 3/10
618/618 [==============================] - 0s 149us/sample - loss: 0.0027 - accuracy: 1.0000 - val_loss: 0.0012 - val_accuracy: 1.0000
Epoch 4/10
618/618 [==============================] - 0s 143us/sample - loss: 0.0024 - accuracy: 1.0000 - val_loss: 0.0013 - val_accuracy: 1.0000
Epoch 5/10
618/618 [==============================] - 0s 150us/sample - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.0011 - val_accuracy: 1.0000
Epoch 6/10
618/618 [==============================] - 0s 151us/sample - loss: 0.0020 - accuracy: 1.0000 - val_loss: 9.8436e-04 - val_accuracy: 1.0000
Epoch 7/10
618/618 [==============================] - 0s 195us/sample - lo